# MLP: Image Classification on MNIST

In [4]:
import tensorflow as tf
import tensorlayer as tl

2023-10-23 11:01:24.282630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# loading the MNSIT dataset by TensorLayer
X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=(-1, 784))
# each image in MNIST is originally sized 28x28, i.e. has 784 pixels

[TL] Load or Download MNIST > data/mnist
[TL] data/mnist/train-images-idx3-ubyte.gz
[TL] data/mnist/t10k-images-idx3-ubyte.gz


In [7]:
# build the model
ni = tl.layers.Input([None, 784])  # the input is aligned with the shape of data
# the layers of the MLP is connected one by one
nn = tl.layers.Dropout(keep=0.8)(ni)
nn = tl.layers.Dense(n_units=800, act=tf.nn.relu)(nn)
nn = tl.layers.Dropout(keep=0.5)(nn)
nn = tl.layers.Dense(n_units=800, act=tf.nn.relu)(nn)
nn = tl.layers.Dropout(keep=0.5)(nn)
nn = tl.layers.Dense(n_units=10, act=None)(nn)
# create the model with specified inputs and outputs
network = tl.models.Model(inputs=ni, outputs=nn, name="mlp")

[TL] Input  _inputlayer_1: [None, 784]


[TL] Dropout dropout_1: keep: 0.800000 
[TL] Dense  dense_1: 800 relu
[TL] Dropout dropout_2: keep: 0.500000 
[TL] Dense  dense_2: 800 relu
[TL] Dropout dropout_3: keep: 0.500000 
[TL] Dense  dense_3: 10 No Activation


In [8]:
# define a metric to evaluate the accuracy of the model
# different from the loss function, the metric is NOT ised to backpropagate or update the model
def acc(_logits, y_batch):
    # return np.mean(np.equal(np.argmax(_logits, 1), y_batch))
    return tf.reduce_mean(
        tf.cast(
            tf.equal(
                tf.argmax(_logits, 1),
                tf.convert_to_tensor(y_batch, tf.int64)),
            tf.float32),
        name='accuracy'
    )

In [9]:
# training
tl.utils.fit(
    network,  # the model
    train_op=tf.optimizers.Adam(learning_rate=0.0001),  # the optimizer
    cost=tl.cost.cross_entropy,  # the loss function
    X_train=X_train, y_train=y_train,  # the training set
    acc=acc,  # the metrics to evaluate the accuracy of a model
    batch_size=256,  # the size of mini-batch
    n_epoch=20,  # number of epoch to train
    X_val=X_val, y_val=y_val, eval_train=True  # validation set
)

[TL] Finished! use `tensorboard --logdir=None/` to start tensorboard
[TL] Start training the network ...
[TL] Epoch 1 of 20 took 5.226250s
[TL]    train loss: 0.413133
[TL]    train acc: 0.884716
[TL] Epoch 5 of 20 took 4.255809s
[TL]    train loss: 0.191547
[TL]    train acc: 0.943149
[TL] Epoch 10 of 20 took 3.595602s
[TL]    train loss: 0.122305
[TL]    train acc: 0.962981
[TL] Epoch 15 of 20 took 3.611428s
[TL]    train loss: 0.088144
[TL]    train acc: 0.973498
[TL] Epoch 20 of 20 took 4.129800s
[TL]    train loss: 0.067379
[TL]    train acc: 0.979728
[TL] Total training time: 85.557495s


In [10]:
# testing
tl.utils.test(
    network,  # the model just trained
    acc=acc,  # the metrics to evaluate the accuracy of a model
    X_test=X_test, y_test=y_test,  # testing set
    batch_size=None,  # the size of mini-batch. If None, the whole testing set is fed into the network together, so only set it None when the testing set is small
    cost=tl.cost.cross_entropy  # the loss function
)

[TL] Start testing the network ...


<tf.Tensor: shape=(), dtype=float32, numpy=0.973>

In [11]:
import numpy as np

# evaluation
_logits = tl.utils.predict(network, X_test)
y_pred = np.argmax(_logits, 1)
tl.utils.evaluation(y_test, y_pred, n_classes=10)

[TL] confusion matrix: 
[[ 970    0    1    1    0    1    4    1    2    0]
 [   0 1126    2    1    0    1    2    0    3    0]
 [   5    0 1006    5    2    0    2    7    5    0]
 [   0    0    6  985    0    3    0    8    6    2]
 [   1    0    4    0  961    0    3    1    2   10]
 [   4    0    0   17    1  854    8    1    4    3]
 [   6    3    0    1    4    4  936    0    4    0]
 [   1    7   11    3    1    0    0  991    0   14]
 [   4    1    3    8    4    3    4    4  940    3]
 [   5    6    1   10   13    1    0    5    7  961]]
[TL] f1-score        : [0.98178138 0.98858648 0.97386254 0.96521313 0.97662602 0.97100625
 0.97652582 0.96871945 0.96558808 0.96003996]
[TL] f1-score(macro) : 0.972795
[TL] accuracy-score  : 0.973000


(array([[ 970,    0,    1,    1,    0,    1,    4,    1,    2,    0],
        [   0, 1126,    2,    1,    0,    1,    2,    0,    3,    0],
        [   5,    0, 1006,    5,    2,    0,    2,    7,    5,    0],
        [   0,    0,    6,  985,    0,    3,    0,    8,    6,    2],
        [   1,    0,    4,    0,  961,    0,    3,    1,    2,   10],
        [   4,    0,    0,   17,    1,  854,    8,    1,    4,    3],
        [   6,    3,    0,    1,    4,    4,  936,    0,    4,    0],
        [   1,    7,   11,    3,    1,    0,    0,  991,    0,   14],
        [   4,    1,    3,    8,    4,    3,    4,    4,  940,    3],
        [   5,    6,    1,   10,   13,    1,    0,    5,    7,  961]]),
 array([0.98178138, 0.98858648, 0.97386254, 0.96521313, 0.97662602,
        0.97100625, 0.97652582, 0.96871945, 0.96558808, 0.96003996]),
 0.973,
 0.972794911127844)

In [12]:
# save network weights to a file
network.save_weights('model.MNIST')

[TL] [*] Saving TL weights into model.MNIST
[TL] [*] Saved
